In [88]:
import requests # make http requests
import json # make sense of what the requests return
import pickle # save our data to our computer

import pandas as pd # structure out data
import numpy as np # do a bit of math
import scipy.stats as sci # do a bit more math

from datetime import datetime # make working with dates 1000x easier 
import time # become time lords
import random # create random numbers (probably not needed)

import sys
import requests
from bs4 import BeautifulSoup

In [85]:
url = "https://store.steampowered.com/"

In [86]:
page = requests.get(url)

In [89]:
soup = BeautifulSoup(page.content, 'lxml')    
type(soup)

bs4.BeautifulSoup

In [90]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html class="responsive" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="#171a21" name="theme-color"/>
<title>Welcome to Steam</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/motiva_sans.css?v=Rc2hpzg2Ex3T&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/shared_global.css?v=7osXnMYyq5rM&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/buttons.css?v=6PFqex5UPprb&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/css/v6/store.css?v=9qpnDZtFMn1Y&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com

In [30]:
#########################################################


### TASK -1

In [31]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# customisations - ensure tables show all columns
pd.set_option("max_columns", 100)

In [1]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

In [35]:
url = "https://steamspy.com/api.php"
parameters = {"request": "all"}

# request 'all' from steam spy and parse into dataframe
json_data = get_request(url, parameters=parameters)
steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')

# generate sorted app_list from steamspy data
app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)


# display first few rows
app_list


appid                                     name
0       2570                  Vigil: Blood Bitterness
1      24460            King Arthur: Fallen Champions
2      34275                              Gain Ground
3      34276                               Golden Axe
4      34277  Shinobi III: Return of the Ninja Master
..       ...                                      ...
995  1902120                            Bigfield 2042
996  1903220                           Paint To Pixel
997  1903480                            Banana Hunter
998  1906150                            Pixel Trouble
999  1908650                              Blastocitos

[1000 rows x 2 columns]

In [38]:
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

In [39]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

In [95]:
def parse_steam_request(appid, name):
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data


# Set file parameters
download_path = "Downloads"
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    end=10,
    batchsize=5
)

Starting at index 0:

Exported lines 0-4 to steam_app_data.csv. Batch 0 time: 0:00:12 (avg: 0:00:12, remaining: 0:00:12)
Exported lines 5-9 to steam_app_data.csv. Batch 1 time: 0:00:10 (avg: 0:00:11, remaining: 0:00:00)

Processing batches complete. 10 apps written


### TASK - 2 

In [54]:
pwd


'C:\\Users\\Manish'

In [60]:
pd.read_csv('C:\\Users\\Manish\steam_app_data.csv')

type                                          name  steam_appid  \
0  game                      Vigil: Blood Bitterness™         2570   
1  game                 King Arthur: Fallen Champions        24460   
2  game                                  Gain Ground™        34275   
3  game                                   Golden Axe™        34276   
4  game      Shinobi™ III: Return of the Ninja Master        34277   
5  game                               Sonic 3D Blast™        34278   
6  game                            All Aspect Warfare        36910   
7  game  Hamilton's Great Adventure - Retro Fever DLC        42147   
8  game                 Hearts of Iron III: Semper Fi        42900   
9  game                          Defense of the Oasis        45200   

   required_age  is_free  controller_support                    dlc  \
0             0    False                 NaN                    NaN   
1             0    False                 NaN                    NaN   
2             0    False                 NaN                    NaN   
3             0    False                 NaN                    NaN   
4             0    False                 NaN                    NaN   
5             0    False                 NaN                    NaN   
6             0    False                 NaN                    NaN   
7             0    False                 NaN                    NaN   
8             0    False                 NaN  [42905, 42903, 42902]   
9             0    False                 NaN                    NaN   

                                detailed_description  \
0  <p>Vigil: Blood Bitterness plunges you into th...   
1  The great King Arthur has crushed his adversar...   
2  The battle-simulation game has gone haywire!<b...   
3  <h1>Just Updated</h1><p>All SEGA MegaDrive/Gen...   
4  Joe Musashi’s sworn enemy, the Neo Zeed are se...   
5  Dr. Eggman (AKA Dr. Robotnik) discovers unusua...   
6  An elite strike team — with a thermo-nuclear w...   
7  This map pack contains new levels that bring b...   
8  It’s time to dust off the field marshal's bato...   
9  The award-winning cult classic has emerged fro...   

                                      about_the_game  \
0  <p>Vigil: Blood Bitterness plunges you into th...   
1  The great King Arthur has crushed his adversar...   
2  The battle-simulation game has gone haywire!<b...   
3  The land of Yuria has been invaded and is now ...   
4  Joe Musashi’s sworn enemy, the Neo Zeed are se...   
5  Dr. Eggman (AKA Dr. Robotnik) discovers unusua...   
6  An elite strike team — with a thermo-nuclear w...   
7  This map pack contains new levels that bring b...   
8  It’s time to dust off the field marshal's bato...   
9  The award-winning cult classic has emerged fro...   

                                   short_description  fullgame  \
0  Vigil: Blood Bitterness plunges you into the d...       NaN   
1  An all-new standalone expansion to the hit RPG...       NaN   
2  The battle-simulation game has gone haywire! A...       NaN   
3  The land of Yuria has been invaded and is now ...       NaN   
4  Joe Musashi’s sworn enemy, the Neo Zeed are se...       NaN   
5  Dr. Eggman (AKA Dr. Robotnik) discovers unusua...       NaN   
6  All Aspect Warfare is a thrilling planetary co...       NaN   
7  This map pack contains new levels that bring b...       NaN   
8  It’s time to dust off the field marshal's bato...       NaN   
9  One part Minesweeper, one part Civilization. W...       NaN   

                                 supported_languages  \
0                                            English   
1           English, French, German, Spanish - Spain   
2                                            English   
3                                            English   
4                                            English   
5                                            English   
6                                            English   
7  English, French, Italian, German, Span

In [96]:
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}
    
    json_data = get_request(url, parameters)
    return json_data


# set files and columns
download_path = "Downloads"
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'

steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
    end=250,
    batchsize=5,
    pause=0.3
)

Starting at index 0:

Exported lines 0-4 to steamspy_data.csv. Batch 0 time: 0:00:07 (avg: 0:00:07, remaining: 0:05:32)
Exported lines 5-9 to steamspy_data.csv. Batch 1 time: 0:00:07 (avg: 0:00:07, remaining: 0:05:24)
Exported lines 10-14 to steamspy_data.csv. Batch 2 time: 0:00:07 (avg: 0:00:07, remaining: 0:05:17)
Exported lines 15-19 to steamspy_data.csv. Batch 3 time: 0:00:07 (avg: 0:00:07, remaining: 0:05:11)
Exported lines 20-24 to steamspy_data.csv. Batch 4 time: 0:00:07 (avg: 0:00:07, remaining: 0:05:04)
Exported lines 25-29 to steamspy_data.csv. Batch 5 time: 0:00:07 (avg: 0:00:07, remaining: 0:04:57)
Exported lines 30-34 to steamspy_data.csv. Batch 6 time: 0:00:07 (avg: 0:00:07, remaining: 0:04:50)
Exported lines 35-39 to steamspy_data.csv. Batch 7 time: 0:00:07 (avg: 0:00:07, remaining: 0:04:43)
Exported lines 40-44 to steamspy_data.csv. Batch 8 time: 0:00:07 (avg: 0:00:07, remaining: 0:04:36)
Exported lines 45-49 to steamspy_data.csv. Batch 9 time: 0:00:07 (avg: 0:00:07, re

In [97]:
df=pd.read_csv('C:\\Users\\Manish\steamspy_data.csv')
df

appid                                     name  \
0      2570                  Vigil: Blood Bitterness   
1     24460            King Arthur: Fallen Champions   
2     34275                              Gain Ground   
3     34276                               Golden Axe   
4     34277  Shinobi III: Return of the Ninja Master   
..      ...                                      ...   
245  735460                          Highway Madness   
246  738080                Creatio Ex Nihilo: Aition   
247  738190                                    Morth   
248  739240              The Ghosts of Hackney Mills   
249  744610                     TAD: That Alien Dude   

                       developer                    publisher  score_rank  \
0                      Freegamer              HeR Interactive         NaN   
1                   NeocoreGames          Paradox Interactive         NaN   
2                           SEGA                         SEGA         NaN   
3                           SEGA                         SEGA         NaN   
4                           SEGA                         SEGA         NaN   
..                           ...                          ...         ...   
245                   Game Cooks                   Game Cooks         NaN   
246                  Shoho Games                  Shoho Games         NaN   
247    Hidden Crawlspace Studios    Hidden Crawlspace Studios         NaN   
248  Sapphire Dragon Productions  Sapphire Dragon Productions         NaN   
249  Modern Aeon, Anderson Evans                  Modern Aeon         NaN   

     positive  negative  userscore              owners  average_forever  \
0          29       117          0   50,000 .. 100,000                1   
1          18        28          0   50,000 .. 100,000                0   
2           5         0          0   50,000 .. 100,000                0   
3          18         1          0    20,000 .. 50,000                0   
4           5         0          0   50,000 .. 100,000                0   
..        ...       ...        ...                 ...              ...   
245        27        11          0    20,000 .. 50,000                0   
246         4        11          0   50,000 .. 100,000                0   
247         3         1          0  100,000 .. 200,000                0   
248        11         6          0   50,000 .. 100,000                0   
249         1         4          0  100,000 .. 200,000                0   

     average_2weeks  median_forever  median_2weeks  price  initialprice  \
0                 0               1              0      0             0   
1                 0               0              0    499           499   
2                 0               0              0     99            99   
3                 0               0              0     99            99   
4                 0               0              0     99            99   
..              ...             ...            ...    ...           ...   
245               0               0              0      0             0   
246               0               0              0     49            99   
247               0               0              0      0             0   
248               0               0              0    199           199   
249               0               0              0    499           499   

     discount                                      languages  \
0           0                                        English   
1           0       English, French, German, Spanish - Spain   
2           0                                        English   
3           0                                        English   
4           0                                        English   
..        ...                                            ...   
245         0                                        English   
246        51  English, Spanish - Spain, Portuguese - Brazil   
247         0                 

### TASK - 3

In [98]:
gen= df.groupby(['genre'])['name'].count()
gen

genre
Action                                                           8
Action, Adventure                                                5
Action, Adventure, Casual                                        1
Action, Adventure, Casual, Free to Play, Indie, RPG, Strategy    1
Action, Adventure, Casual, Indie                                 6
                                                                ..
Strategy, Early Access                                           1
Utilities                                                        1
Utilities, Early Access                                          1
Utilities, Video Production                                      1
Violent, Gore, Adventure, Indie                                  1
Name: name, Length: 110, dtype: int64

Action, Adventure, RPG, Casual, Strategy are the most popular genre (games). 

### TASK -4

In [101]:
res=df.sort_values(by=['initialprice'],ascending=[False])
eg = res[['appid','name','developer','publisher','owners','initialprice','genre']]
eg

appid                                             name  \
21   279280                                         Ohmicide   
147  555970                                           Awaken   
24   301220                                Legends of Persia   
75   397820                                  Let's Sing 2016   
201  638060                                  Majestic Trials   
..      ...                                              ...   
59   372190                          Astro Lords: Oort Cloud   
212  658420                      Agents of Aggro City Online   
135  530550  Bubble Jungle  Super Chameleon Platformer World   
214  659630                                     Tiny Toyfare   
0      2570                          Vigil: Blood Bitterness   

                      developer                   publisher  \
21                    Ohm Force                   Ohm Force   
147      Blueprint Reality Inc.      Blueprint Reality Inc.   
24          Sourena Game Studio             Plug In Digital   
75                 Voxler Games                 Ravenscourt   
201  Alpha Strike Games Limited  Alpha Strike Games Limited   
..                          ...                         ...   
59                   Aratog LLC                   BISBOG SA   
212                        none                        none   
135                         NaN            Eternity Studios   
214         Super Suite Studios         Super Suite Studios   
0                     Freegamer             HeR Interactive   

                 owners  initialprice  \
21    50,000 .. 100,000         12499   
147    20,000 .. 50,000          2999   
24    50,000 .. 100,000          1999   
75     20,000 .. 50,000          1999   
201   50,000 .. 100,000          1499   
..                  ...           ...   
59     20,000 .. 50,000             0   
212  100,000 .. 200,000             0   
135    20,000 .. 50,000             0   
214   50,000 .. 100,000             0   
0     50,000 .. 100,000             0   

                                                 genre  
21                                    Audio Production  
147             Adventure, Casual, Indie, Early Access  
24                       Action, Adventure, Indie, RPG  
75                                              Casual  
201                    Adventure, Indie, RPG, Strategy  
..                                                 ...  
59                                        Free to Play  
212  Adventure, Casual, Indie, Massively Multiplaye...  
135                                                NaN  
214              Action, Free to Play, Indie, Strategy  
0                                           Indie, RPG  

[250 rows x 7 columns]

### TASK - 5

In [102]:
top =  df[['name','publisher','initialprice','genre']]
top

name                    publisher  \
0                    Vigil: Blood Bitterness              HeR Interactive   
1              King Arthur: Fallen Champions          Paradox Interactive   
2                                Gain Ground                         SEGA   
3                                 Golden Axe                         SEGA   
4    Shinobi III: Return of the Ninja Master                         SEGA   
..                                       ...                          ...   
245                          Highway Madness                   Game Cooks   
246                Creatio Ex Nihilo: Aition                  Shoho Games   
247                                    Morth    Hidden Crawlspace Studios   
248              The Ghosts of Hackney Mills  Sapphire Dragon Productions   
249                     TAD: That Alien Dude                  Modern Aeon   

     initialprice                                 genre  
0               0                            Indie, RPG  
1             499                         RPG, Strategy  
2              99                              Strategy  
3              99                                Action  
4              99                                Action  
..            ...                                   ...  
245             0    Casual, Free to Play, Early Access  
246            99                 Adventure, Indie, RPG  
247             0         Action, Adventure, Indie, RPG  
248           199  Adventure, Casual, Indie, Simulation  
249           499                    Casual, Indie, RPG  

[250 rows x 4 columns]

### TASK -6

In [100]:
image_urls=[]
url = 'https://store.steampowered.com/'

# get contents from url
content = requests.get(url).content

# get soup
soup = BeautifulSoup(content,'lxml') # choose lxml parser

# find the tag : <img ... >
image_tags = soup.findAll('img')

# print out image urls
for image_tag in image_tags:
    image_urls.append(image_tag.get('src'))
image_urls

['https://store.akamai.steamstatic.com/public/shared/images/responsive/logo_valve_footer.png',
 'https://store.akamai.steamstatic.com/public/shared/images/responsive/header_menu_hamburger.png',
 'https://store.akamai.steamstatic.com/public/shared/images/responsive/header_logo.png',
 'https://store.akamai.steamstatic.com/public/shared/images/header/logo_steam.svg?t=962016',
 'https://store.akamai.steamstatic.com/public/images//gift/steamcards_promo_02.png',
 'https://store.akamai.steamstatic.com/public/images/blank.gif',
 'https://store.akamai.steamstatic.com/public/images/blank.gif',
 'https://store.akamai.steamstatic.com/public/images/mobile/ResponsiveChevron.svg',
 'https://cdn.akamai.steamstatic.com/steam/spotlights/c05da286ed0d2f225de68f1d/spotlight_image_english.jpg?t=1646259014',
 'https://cdn.akamai.steamstatic.com/steam/spotlights/f49c5505b0f71707eed78cb4/spotlight_image_english.jpg?t=1645744745',
 'https://cdn.akamai.steamstatic.com/steam/apps/727130/header_alt_assets_2.jpg?t=